### Carlo.ai's Fastcore Quantom Blogpost

Jeremy shared this blogpost [fastcore-quantum](https://carlo.ai/posts/fastcore-quantum) and we should check they keep working.

Below I've extracted the code from the blog and added assertions for automated checking. This will be useful when creating the new Transform implementation.

In [ ]:
import numpy as np
from numpy.testing import assert_almost_equal

from fasttransform.core import Transform
from fasttransform.pipeline import Pipeline
# from fastcore.transform import Transform, Pipeline

class S(Transform):
    def encodes(self, x): return x ** 2
    def decodes(self, x): return x ** 0.5
            
assert S()(10) == 100
assert S().decode(100) == 10
assert S().decode(S()(10)) == 10

In [ ]:
square = Transform(lambda x: x ** 2)
assert square(10) == 100
assert square.decode(100) == 100

In [ ]:
@Transform
def square(x): return x ** 2
square(10) # 100
type(square) # <class 'fastcore.transform.Transform'>

fasttransform.core.Transform

In [ ]:
class MultiS(Transform):
    def encodes(self, x: int | float | complex | tuple): return x**2
    def encodes(self, x: list): return [x**2 for x in x]
    def decodes(self, x: int | float | complex | tuple): return x**0.5
    def decodes(self, x: list): return [x**0.5 for x in x]

ms = MultiS()
ms

NameError: name '_get_name' is not defined

In [ ]:
# Lists
# By default, Transform processes lists as a whole
# 2nd encodes method is called
assert ms([1, 2, 3]) == [1, 4, 9]

In [ ]:
# 2nd decodes method is called
assert ms.decode([1, 4, 9]) == [1.0, 2.0, 3.0]                     

In [ ]:
# Tuples
# By default, Transform processes tuples elementwise
# 1st decodes method is called
assert ms((1, 2, 3)) == (1, 4, 9)

In [ ]:
# 1st decodes method is called
assert ms.decode((1, 4, 9)) == (1.0, 2.0, 3.0)

In [ ]:
# Complex numbers
# 1st encodes method is called on complex number
assert ms(10.0j) == (-100+0j)

In [ ]:
# 1st decodes method is called on complex number
assert ms.decode(ms(10.0j)) == (6.123233995736766e-16+10j)

In [ ]:
class S(Transform):
    "Square a number. Reverse is square root."
    def encodes(self, x): return x ** 2
    def decodes(self, x): return x ** 0.5
    
class A(Transform):
    "Add 1. Reverse is subtract 1."
    def encodes(self, x): return x + 1
    def decodes(self, x): return x - 1

pipe = Pipeline([S(), A()])
assert pipe(10) == 101 # 10**2 + 1 = 101
assert pipe.decode(10) == 3.0 # (10 - 1)**0.5 = 3.0
assert pipe.decode(pipe(10)) == 10 # (10**2 + 1 - 1)**0.5 = 10

In [ ]:
class _Q(Transform):
    "Base transform for quantum gates"
    def encodes(self, x): return x @ self.gate
    def decodes(self, x): return x @ self.gate.conj().T
                     
class I(_Q):
    "Identity gate. Does nothing."
    gate = np.array([[1, 0], 
                     [0, 1]])
    
class X(_Q):
    "X (NOT) gate. Flips from |0> to |1> and vice versa."
    gate = np.array([[0, 1], 
                     [1, 0]])
    
class H(_Q):
    "Hadamard (Superposition) gate. Turns a qubit into a superposition."
    gate = np.array([[1, 1], 
                     [1, -1]]) / np.sqrt(2)

In [ ]:
zero_state = [1+0j, 0+0j] # Basis state |0>
superposition_state = np.array([0.5+0.5j, 0.5-0.5j])  # Complex superposition

In [ ]:
# Identity operation
i = I() 
assert_almost_equal(i(zero_state), np.array([1.+0.j, 0.+0.j])) # (|0>)
assert_almost_equal(i.decode(zero_state), np.array([1.+0.j, 0.+0.j]))  # (|0>)                     

In [ ]:
# X (NOT) operation 
x = X()
assert_almost_equal(x(zero_state), np.array([0+0j, 1+0j]))  # (|1>)
assert_almost_equal(x.decode(x(zero_state)), np.array([1.+0.j, 0.+0.j]))  # (|0>)
assert_almost_equal(x(superposition_state), np.array([0.5-0.5j, 0.5+0.5j]))  # (flips sign of complex part)

In [ ]:
# Hadamard gate tests
h = H()
assert_almost_equal(h(zero_state), np.array([0.707+0.j, 0.707+0.j]), decimal=3)  # (superposition)
assert_almost_equal(h(superposition_state), np.array([0.707+0.j, 0.+0.707j]), decimal=3)  # (phase state)
assert_almost_equal(h.decode(h(superposition_state)), np.array([0.5+0.5j, 0.5-0.5j]))  # (complex superposition)

In [ ]:
class M(Transform):
    "Turn a quantum statevector into a probability distribution"
    def encodes(self, x): return np.abs(x)**2
    def decodes(self, x): return NotImplementedError("No inverse exists for absolute value.")

class Samp(Transform):
    "Sample from a probability distribution"
    def encodes(self, x): return format(np.random.choice(len(x), p=x), f'0{int(np.log2(len(x)))}b')
    def decodes(self, x): return NotImplementedError("Sampling is not reversible.")

m = M()
samp = Samp()

In [ ]:
# Sampling from zero state (|0>)
zero_state = [1+0j, 0+0j]
mzs = m(zero_state) # Transforms [1+0j, 0+0j] -> [1, 0]

assert_almost_equal(mzs, np.array([1+0j,0+0j]))                     
assert samp(mzs) == '0'

In [ ]:
# Sampling from equal superposition
equal_superposition = [0.707, 0.707]
mes = m(equal_superposition) # Transforms [0.707, 0.707] -> [0.5, 0.5] (A coin flip. i.e. Bernoulli distribution))
mes

array([0.499849, 0.499849])

In [ ]:
mes = mes / mes.sum()  # not in original blog but needed as otherwises numers dont add to 1 because of rounding
assert_almost_equal(mes, np.array([0.5,0.5]))
assert samp(mes) in '01' # 0 or 1 with equal probability

**⁉️ Question: Check if I'm missing something that I need this normalization**

In [ ]:
# Sampling from complex superposition
complex_superposition = [0.5+0.5j, 0.5-0.5j]
mcs = m(complex_superposition) # Transforms [0.5+0.5j, 0.5-0.5j] -> [0.5, 0.5] (A coin flip. i.e. Bernoulli distribution)
assert_almost_equal(mcs, np.array([0.5,0.5]))

In [ ]:
assert samp(mcs) in '01'  # Result is 0 or 1 with equal probability

In [ ]:
qc = Pipeline([X(), H(), I(), M(), Samp()])
# X transforms [1, 0] -> [0, 1]
# H transforms [0, 1] -> [0.707+0j, -0.707+0j]
# I transforms [0.707+0j, -0.707+0j] -> [0.707+0j, -0.707+0j]
# M transforms [0.707+0j, -0.707+0j] -> [0.5, 0.5]
# Samp samples from random distribution [0.5, 0.5]
assert qc(zero_state) in '01' # 0 or 1 with equal probability

In [ ]:
class Concat(Transform):
    "Combine single qubit gates into a multi-qubit gate"
    def __init__(self, gates): self.gates = gates
    # Concatenate 2 or more gates
    def encodes(self, x): return x @ np.kron(*[g.gate for g in self.gates])
    # Reverse propagation for all gates
    def decodes(self, x):
        for g in reversed(self.gates): x = x @ np.kron(g.gate.conj().T, np.eye(len(x) // g.gate.shape[0]))
        return x

In [ ]:
class CNOT(_Q):
    "Controlled NOT gate"
    def __init__(self): self.gate = np.array([[1, 0, 0, 0], 
                                              [0, 1, 0, 0], 
                                              [0, 0, 0, 1], 
                                              [0, 0, 1, 0]])
                     
two_qubit_zero_state = np.array([1+0j, 0+0j, 0+0j, 0+0j]) # |00>
qc = Pipeline([Concat([H(), I()]), CNOT(), M(), Samp()])  # ADAPTATION, had a trailing comma, probably typo
# Concat([H(), I()]) transforms [1, 0, 0, 0] -> [0.707, 0, 0.707, 0]
# CNOT() transforms [0.707, 0, 0.707, 0] -> [0.707, 0, 0, 0.707]
# M() transforms [0.707, 0, 0, 0.707] -> [0.5, 0, 0, 0.5]
# Samp() samples from [0.5, 0, 0, 0.5] (50% chance at 00 and 50% chance at 3, which is 11 in binary)
assert qc(two_qubit_zero_state) in ('00','11') # 00 or 11 with equal probability